In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
from PIL import Image
import pydicom
from sklearn.model_selection import train_test_split
import nibabel as nib
import seaborn as sns
from tqdm import tqdm

import pytorch_lightning as L
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.optim as optim

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class Config:
    SEED = 101
    BATCH_SIZE = 16
    MAX_EPOCHS = 5
    LR=0.005

config = Config()
random.seed(config.SEED)
np.random.seed(config.SEED)

In [3]:
original = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
test_or = pd.read_parquet('/kaggle/input/rsna-2023-abdominal-trauma-detection/test_dicom_tags.parquet')

In [4]:
df_injuried = original[original['any_injury']==1]
injuried = df_injuried['patient_id']

In [5]:
#Creation of the path of the DICOM images
root_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
patients = injuried

serial_path = {'patient_id':[],
               'serial_id':[],
               'serial_path':[]
              }

serials = {'patient_path':[],
           'patient': []
          }
# Populate serials['patient_path'] first
for patient_id in patients:
    patient_path = os.path.join(root_dir, str(patient_id))
    serials['patient_path'].append(patient_path)
    serials['patient'].append(patient_id)
    
serial_list = list(serials.values())
patient_paths, patient_ids = serial_list

#Now, let's create entries in serial_path for each serial ID
for patient_path, patient_id in zip(patient_paths, patient_ids):
    serial_ids = os.listdir(patient_path)
    for serial_id in serial_ids:
        serial_path['patient_id'].append(patient_id)
        serial_path['serial_id'].append(serial_id)
        serial_path['serial_path'].append(os.path.join(patient_path,serial_id))

df_serials = pd.DataFrame(serial_path)

image_paths = {'patient_id':[],'serial_id':[],'serial_path':[],'image_path':[], 'image':[] }

patient_path_dir = df_serials['patient_id']
serial_id_dir = df_serials['serial_id']
serial_path_dir = df_serials['serial_path']

for patient_id, serial_id, serial_path in zip(patient_path_dir, serial_id_dir, serial_path_dir):
    images = os.listdir(serial_path)
    for image in images:
        image_paths['patient_id'].append(patient_id)
        image_paths['serial_id'].append(serial_id)
        image_paths['image_path'].append(os.path.join(serial_path,image))
        image_paths['serial_path'].append(serial_path)
        image_paths['image'].append(image)
        
image_paths_df = pd.DataFrame(image_paths)

image_paths_df['image'] = image_paths_df['image'].str.replace('.dcm', '').astype(int)

In [6]:
#Separate the serial_id with more than 50 images, less than 50 images and exactly 50 images
pidf = pd.DataFrame()
lidf = pd.DataFrame()
midf = pd.DataFrame()
num_slices = 50
for serial, group in image_paths_df.groupby('serial_id'):
    if len(group) == num_slices:
        group_sorted = group.sort_values('image')
        pidf = pd.concat([pidf,group_sorted],axis=0,ignore_index=True)
    elif len(group) > num_slices:
        group_sorted = group.sort_values('image')
        midf = pd.concat([midf,group_sorted],axis=0,ignore_index=True)
    else:
        group_sorted = group.sort_values('image')
        lidf = pd.concat([lidf,group_sorted],axis=0,ignore_index=True)

In [7]:
new_more_df = pd.DataFrame()  
for serial_id, group in midf.groupby('serial_id'):
    total_rows = group.shape[0]
    step_size = max(total_rows // num_slices, 1)  
    selected_rows = group.iloc[::step_size][:num_slices].reset_index(drop=True)
    new_more_df = pd.concat([new_more_df, selected_rows], ignore_index=True)

In [8]:
new_less_df = pd.DataFrame()
for serial, group in lidf.groupby('serial_id'):
    length = len(group)
    intergers = num_slices//length
    rest = num_slices%length
    group_df = pd.concat([group] * intergers, axis=0)
    if rest >0:
        step_size = max(length // rest, 1)
        selected_rows = group.iloc[::step_size][:rest]
        inter_df = pd.concat([group_df,selected_rows], axis=0)
    new_less_df = pd.concat([new_less_df,inter_df], axis=0)
    new_less_df = new_less_df.sort_index()

In [9]:
total_dfs = pd.concat([pidf,new_less_df,new_more_df],axis=0,ignore_index=True)
df_total = pd.merge(total_dfs,df_injuried, on='patient_id')

In [10]:
from sklearn.model_selection import train_test_split
df_s = pd.merge(df_serials, df_injuried,on = 'patient_id')
X = df_s['serial_id']
y = df_s[['bowel_healthy',
          'bowel_injury', 
          'extravasation_healthy', 
          'extravasation_injury',
          #'kidney_healthy', 
          #'kidney_low', 
          #'kidney_high', 
          'liver_healthy',
          #'liver_low', 
          #'liver_high', 
          'spleen_healthy',
          'spleen_low',
          'spleen_high'
         ]]
X_train, X_test, y_train, y_test = train_test_split(
    X,y , test_size=0.2, stratify=y, random_state=101
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train ,y_train , test_size=0.1, stratify=y_train, random_state=101
)

In [11]:
train = df_total[df_total['serial_id'].isin(X_train)]
test = df_total[df_total['serial_id'].isin(X_test)]
val = df_total[df_total['serial_id'].isin(X_val)]

In [12]:
def create_sequences_with_step_image(data: pd.DataFrame, seq_len, step):
    sequences = []
    data_size = len(data)

    for i in tqdm(range(0, data_size - seq_len + 1, step)):
        feature = data.iloc[i: i + seq_len,3]
        label_position = (i+seq_len-1)
        label = data.iloc[label_position,5:-1]
        
        sequences.append([feature,label])
        
    return sequences

seq_len = num_slices

In [13]:
train_s = create_sequences_with_step_image(train,seq_len,seq_len)
test_s = create_sequences_with_step_image(test,seq_len,seq_len)
val_s = create_sequences_with_step_image(val,seq_len,seq_len)

100%|██████████| 103/103 [00:00<00:00, 4066.35it/s]


In [14]:
#Creation of the test sequences
test = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/test_series_meta.csv')
test_pat_id = test['patient_id'].unique()

# Define the root directory where the patient data is located
root_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'

# Define the list of patient IDs you want to use (replace with your selected IDs)
selected_patient_ids = test_pat_id 

# Initialize empty lists to store data
patient_ids = []
image_paths = []

# Traverse the directories and collect the data
for patient_id in selected_patient_ids:
    patient_dir = os.path.join(root_dir, str(patient_id))
    if os.path.exists(patient_dir):
        for serial_id in os.listdir(patient_dir):
            serial_dir = os.path.join(patient_dir, serial_id)
            if os.path.isdir(serial_dir):
                # List all .dcm files in the serial_id folder
                image_files = [filename for filename in os.listdir(serial_dir) if filename.endswith('.dcm')]
                
                # Calculate the step size for evenly spaced selection
                total_images = len(image_files)
                if total_images <= num_slices:
                    step_size = 1  # If fewer than num_slices images, select all of them
                else:
                    step_size = total_images // num_slices
                
                # Select images in an evenly spaced manner
                selected_images = [image_files[i] for i in range(0, total_images, step_size)][:num_slices]
                
                for filename in selected_images:
                    image_path = os.path.join(serial_dir, filename)
                    patient_ids.append(patient_id)
                    image_paths.append(image_path)

# Create a DataFrame from the collected data
df_test = {'patient_id': patient_ids, 'image_path': image_paths}
df_test = pd.DataFrame(df_test)
df_test_num_slices = pd.DataFrame()
for i in df_test.index:
    df= pd.concat([df_test.iloc[i:i+1,:]]*num_slices,axis=0, ignore_index=True)
    df_test_num_slices = pd.concat([df_test_num_slices,df],axis=0, ignore_index=True)
def create_sequences_with_step_image_inf(data: pd.DataFrame, seq_len, step):
    sequences = []
    data_size = len(data)

    for i in tqdm(range(0, data_size - seq_len + 1, step)):
        feature = data.iloc[i: i + seq_len,1]
        sequences.append([feature])
    return sequences

seq_len = num_slices
test_seq = create_sequences_with_step_image_inf(df_test_num_slices,seq_len,seq_len)

100%|██████████| 3/3 [00:00<00:00, 7307.15it/s]


In [15]:
np.save('train_s.npy', train_s)
np.save('test_s.npy', train_s)
np.save('val_s.npy', train_s)
np.save('test_seq.npy', test_seq)

/opt/conda/lib/python3.10/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [16]:
class SeqToImageDataset(Dataset):

    def __init__(self,sequences):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):   
        sequence, labels = self.sequences[idx]
        sequence_features = []

        for image_path in sequence:
            path = image_path
            ds = pydicom.dcmread(path)
            image = ds.pixel_array
            image = image.astype('int16')
            image = Image.fromarray(image).convert('RGB')
            image = image.resize((224, 224))
            
            t = transforms.Compose([#transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
            image = t(image)
            
        sequence_features.append(image)
        return {'sequence': sequence_features, 'labels': torch.tensor(labels, dtype=torch.float32)}

In [17]:
class SeqToImageDatasetINF(Dataset):

    def __init__(self,sequences):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):   
        sequence = self.sequences[idx]
        sequence_features = []

        for image_path in sequence[0]:
            path = image_path
            ds = pydicom.dcmread(path)
            image = ds.pixel_array
            image = image.astype('int16')
            image = Image.fromarray(image).convert('RGB')
            image = image.resize((224, 224))
            
            t = transforms.Compose([#transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
            image = t(image)
            
        sequence_features.append(image)
        return sequence_features

In [18]:
class CustomDataModule(L.LightningDataModule):
    def __init__(self, train_dataframe, val_dataframe, test_dataframe, 
                 pred_dataframe, 
                 batch_size=config.BATCH_SIZE):
        super().__init__()
        self.train_dataframe = train_dataframe
        self.val_dataframe = val_dataframe
        self.test_dataframe = test_dataframe
        self.pred_dataframe = pred_dataframe

        self.batch_size = batch_size
        
        random.seed(config.SEED)
        np.random.seed(config.SEED)
        torch.manual_seed(config.SEED)

    def setup(self, stage=None):
        self.train_dataset = SeqToImageDataset(self.train_dataframe)
        self.val_dataset = SeqToImageDataset(self.val_dataframe)
        self.test_dataset = SeqToImageDataset(self.test_dataframe)
        self.pred_dataset = SeqToImageDatasetINF(self.pred_dataframe)

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=1
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.val_dataset,
            batch_size=len(self.val_dataframe)//5,
            shuffle=False,
            num_workers=1
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test_dataset,
            batch_size=len(self.test_dataframe)//5,
            shuffle=False,
            num_workers=1
        )

    def predict_dataloader(self):
        return DataLoader(
            dataset=self.pred_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=1
        )

In [19]:
class CNNLSTMImage(L.LightningModule):
    
    def __init__(self, n_feat=50, n_hidden=150,n_layers=1):
        super(CNNLSTMImage, self).__init__()
        self.n_feat = n_feat
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        
        self.features = torchvision.models.resnet18(weights='DEFAULT')
        self.features.fc = nn.Linear(in_features=512, out_features=n_feat, bias = True)


        self.lstm = nn.LSTM(
            input_size = n_feat,
            hidden_size = n_hidden,
            batch_first = False,
            num_layers = n_layers
        )
        
        self.layersB = nn.Sequential(
            nn.GELU(),
            nn.Linear(in_features=n_hidden, out_features=2, bias=True),
        )
        
        self.layersE = nn.Sequential(
            nn.GELU(),
            nn.Linear(in_features=n_hidden, out_features=2, bias=True),
        )
        
        
        self.layersK = nn.Sequential(
            nn.GELU(),
            nn.Linear(in_features=n_hidden, out_features=3, bias=True),
        )
        
        
        self.layersL = nn.Sequential(
            nn.GELU(),
            nn.Linear(in_features=n_hidden, out_features=3, bias=True),
        )
  
        
        self.layersS = nn.Sequential(
            nn.GELU(),
            nn.Linear(in_features=n_hidden, out_features=3, bias=True),
        )
        
    def forward(self, x):
        
        sequence_features = []

        for image in x:
            features = self.features(image)  # Pass each image through the feature extractor
            sequence_features.append(features)

        sequence_features = torch.stack(sequence_features)
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(sequence_features)
        out = hidden[-1]
        
        bowel_logits = self.layersB(out)
        extravasation_logits = self.layersE(out)
        kidney_logits = self.layersK(out)
        liver_logits = self.layersL(out)
        spleen_logits = self.layersS(out)
        
        return {'bowel':bowel_logits,
                'extravasation': extravasation_logits,
                'kidney': kidney_logits,
                'liver': liver_logits,
                'spleen': spleen_logits
               }

In [20]:
device = 'cuda'
#Promissor e funcionando
class LightM(L.LightningModule):
    def __init__(self, model, lr=config.LR):
        super(LightM, self).__init__()
        self.lr = lr
        self.model = model
        
        self.loss_fn1 = nn.BCEWithLogitsLoss()
        self.loss_fn2 = nn.BCEWithLogitsLoss()
        self.loss_fn3 = nn.CrossEntropyLoss()
        self.loss_fn4 = nn.CrossEntropyLoss()
        self.loss_fn5 = nn.CrossEntropyLoss()

    def forward(self, x):

        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch['sequence'], batch['labels']

        # Compute logits for both tasks
        logits = self(inputs)
        bowel_logits = logits['bowel']
        extravasation_logits = logits["extravasation"]
        kidney_logits = logits['kidney']
        liver_logits = logits['liver']
        spleen_logits = logits['spleen']

        # Compute losses and accuracies for both tasks
        bowel_loss = self.loss_fn1(bowel_logits, labels[:,0:2])
        extravasation_loss = self.loss_fn2(extravasation_logits, labels[:,2:4])
        kidney_loss = self.loss_fn3(kidney_logits, labels[:,4:7])
        liver_loss = self.loss_fn4(liver_logits, labels[:,7:10])
        spleen_loss = self.loss_fn5(spleen_logits, labels[:,10:])
        
        # Compute the total loss (you can weigh the losses if needed)
        total_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss

        self.log('train_bowel_loss', bowel_loss)
        self.log('train_extravasation_loss', extravasation_loss)
        self.log('train_kidney_loss', kidney_loss)
        self.log('train_liver_loss', liver_loss)
        self.log('train_spleen_loss',spleen_loss)
        self.log('train_loss',total_loss, prog_bar=True, logger=True)
        

        #preds
        bowel_pred = torch.argmax(bowel_logits, dim=1)
        extravasation_pred = torch.argmax(extravasation_logits, dim=1)
        kidney_pred = torch.argmax(kidney_logits, dim=1) 
        liver_pred = torch.argmax(liver_logits, dim=1)
        spleen_pred = torch.argmax(spleen_logits, dim=1)
        #total_pred = torch.cat([bowel_pred, extravasation_pred, kidney_pred, liver_pred, spleen_pred], dim=1)
        acc1 =torchmetrics.Accuracy(task='binary').to(device)
        acc2 =torchmetrics.Accuracy(task='binary').to(device)
        acc3 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc4 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc5 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        accuracy1 = acc1(bowel_pred,torch.argmax(labels[:,0:2], dim=1))
        accuracy2 = acc2(extravasation_pred,torch.argmax(labels[:,2:4], dim=1))
        accuracy3 = acc3(kidney_pred,torch.argmax(labels[:,4:7], dim=1))
        accuracy4 = acc4(liver_pred,torch.argmax(labels[:,7:10], dim=1))
        accuracy5 = acc5(spleen_pred,torch.argmax(labels[:,10:], dim=1))
        acc = (accuracy1 + accuracy2 + accuracy3 + accuracy4 + accuracy5)/5
        self.log('train_bowel_acc', accuracy1)
        self.log('train_extravasation_acc', accuracy2)
        self.log('train_kidney_acc', accuracy3)
        self.log('train_liver_acc', accuracy4)
        self.log('train_spleen_acc',accuracy5)
        self.log('train_acc',acc, prog_bar=True, logger=True)

        return total_loss
    
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch['sequence'], batch['labels']

        # Compute logits for both tasks
        logits = self(inputs)
        bowel_logits = logits['bowel']
        extravasation_logits = logits["extravasation"]
        kidney_logits = logits['kidney']
        liver_logits = logits['liver']
        spleen_logits = logits['spleen']

        # Compute losses and accuracies for both tasks
        bowel_loss = self.loss_fn1(bowel_logits, labels[:,0:2])
        extravasation_loss = self.loss_fn2(extravasation_logits, labels[:,2:4])
        kidney_loss = self.loss_fn3(kidney_logits, labels[:,4:7])
        liver_loss = self.loss_fn4(liver_logits, labels[:,7:10])
        spleen_loss = self.loss_fn5(spleen_logits, labels[:,10:])


        # Compute the total loss (you can weigh the losses if needed)
        total_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss

        self.log('val_bowel_loss', bowel_loss)
        self.log('val_extravasation_loss', extravasation_loss)
        self.log('val_kidney_loss', kidney_loss)
        self.log('val_liver_loss', liver_loss)
        self.log('val_spleen_loss',spleen_loss)
        self.log('val_loss',total_loss, prog_bar=True, logger=True)
        
        #preds
        bowel_pred = torch.argmax(bowel_logits, dim=1)
        extravasation_pred = torch.argmax(extravasation_logits, dim=1)
        kidney_pred = torch.argmax(kidney_logits, dim=1) 
        liver_pred = torch.argmax(liver_logits, dim=1)
        spleen_pred = torch.argmax(spleen_logits, dim=1)
        #total_pred = torch.cat([bowel_pred, extravasation_pred, kidney_pred, liver_pred, spleen_pred], dim=1)
        acc1 =torchmetrics.Accuracy(task='binary').to(device)
        acc2 =torchmetrics.Accuracy(task='binary').to(device)
        acc3 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc4 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc5 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        accuracy1 = acc1(bowel_pred,torch.argmax(labels[:,0:2], dim=1))
        accuracy2 = acc2(extravasation_pred,torch.argmax(labels[:,2:4], dim=1))
        accuracy3 = acc3(kidney_pred,torch.argmax(labels[:,4:7], dim=1))
        accuracy4 = acc4(liver_pred,torch.argmax(labels[:,7:10], dim=1))
        accuracy5 = acc5(spleen_pred,torch.argmax(labels[:,10:], dim=1))
        acc = (accuracy1 + accuracy2 + accuracy3 + accuracy4 + accuracy5)/5
        self.log('val_bowel_acc', accuracy1)
        self.log('val_extravasation_acc', accuracy2)
        self.log('val_kidney_acc', accuracy3)
        self.log('val_liver_acc', accuracy4)
        self.log('val_spleen_acc',accuracy5)
        self.log('val_acc',acc, prog_bar=True, logger=True)
        
        return total_loss
    
    def test_step(self, batch, batch_idx):
        inputs, labels = batch['sequence'], batch['labels']

        # Compute logits for both tasks
        logits = self(inputs)
        bowel_logits = logits['bowel']
        extravasation_logits = logits["extravasation"]
        kidney_logits = logits['kidney']
        liver_logits = logits['liver']
        spleen_logits = logits['spleen']

        # Compute losses and accuracies for both tasks
        bowel_loss = self.loss_fn1(bowel_logits, labels[:,0:2])
        extravasation_loss = self.loss_fn2(extravasation_logits, labels[:,2:4])
        kidney_loss = self.loss_fn3(kidney_logits, labels[:,4:7])
        liver_loss = self.loss_fn4(liver_logits, labels[:,7:10])
        spleen_loss = self.loss_fn5(spleen_logits, labels[:,10:])


        # Compute the total loss (you can weigh the losses if needed)
        total_loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss

        self.log('test_bowel_loss', bowel_loss)
        self.log('test_extravasation_loss', extravasation_loss)
        self.log('test_kidney_loss', kidney_loss)
        self.log('test_liver_loss', liver_loss)
        self.log('test_spleen_loss',spleen_loss)
        self.log('test_loss',total_loss, prog_bar=True, logger=True)
        
        #preds
        bowel_pred = torch.argmax(bowel_logits, dim=1)
        extravasation_pred = torch.argmax(extravasation_logits, dim=1)
        kidney_pred = torch.argmax(kidney_logits, dim=1) 
        liver_pred = torch.argmax(liver_logits, dim=1)
        spleen_pred = torch.argmax(spleen_logits, dim=1)
        #total_pred = torch.cat([bowel_pred, extravasation_pred, kidney_pred, liver_pred, spleen_pred], dim=1)
        acc1 =torchmetrics.Accuracy(task='binary').to(device)
        acc2 =torchmetrics.Accuracy(task='binary').to(device)
        acc3 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc4 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        acc5 =torchmetrics.Accuracy(task='multiclass', num_classes=3).to(device)
        accuracy1 = acc1(bowel_pred,torch.argmax(labels[:,0:2], dim=1))
        accuracy2 = acc2(extravasation_pred,torch.argmax(labels[:,2:4], dim=1))
        accuracy3 = acc3(kidney_pred,torch.argmax(labels[:,4:7], dim=1))
        accuracy4 = acc4(liver_pred,torch.argmax(labels[:,7:10], dim=1))
        accuracy5 = acc5(spleen_pred,torch.argmax(labels[:,10:], dim=1))
        acc = (accuracy1 + accuracy2 + accuracy3 + accuracy4 + accuracy5)/5
        self.log('test_bowel_acc', accuracy1)
        self.log('test_extravasation_acc', accuracy2)
        self.log('test_kidney_acc', accuracy3)
        self.log('test_liver_acc', accuracy4)
        self.log('test_spleen_acc',accuracy5)
        self.log('test_acc',acc, prog_bar=True, logger=True)
        
        return total_loss
    
    def prediction_step(self, batch, batch_idx):
        inputs = batch

        logits = self(inputs)

        return logits

#    def configure_optimizers(self):
#        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
#        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=4)
#        return optimizer
    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.lr)
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=57, eta_min = 0.0005) # New!

        return {
            "optimizer": opt,
            "lr_scheduler": {
                "scheduler": sch,
                "monitor": "train_loss",
                "interval": "step", # step means "batch" here, default: epoch   # New!
                "frequency": 1, # default
            },
        }

In [21]:
module = CNNLSTMImage()
model = LightM(model=module, lr=config.LR)
logger_v1 = TensorBoardLogger(save_dir="/kaggle/working/", name="logs_v1")
callback_v1 = [ModelCheckpoint(save_top_k=1,
                             verbose=True,
                             monitor='val_loss',
                             save_last = True,
                             mode='min',
                             filename='best_model',
                             dirpath = logger_v1.log_dir,
                              ),
               EarlyStopping(monitor='val_loss',
                             patience=3,
                             mode='min',
                            )
              ]

trainer = L.Trainer(
    max_epochs=config.MAX_EPOCHS,
    callbacks = callback_v1,
    accelerator="gpu",
    logger=logger_v1,
    deterministic=True,
    precision='16-mixed',
    log_every_n_steps=1
)

dm = CustomDataModule(train_s,
                         val_s,
                         test_s,
                         test_seq,
                         batch_size=config.BATCH_SIZE,
                        )

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 248MB/s]


In [22]:
trainer.fit(model,datamodule=dm)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /kaggle/working/logs_v1/version_0 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [23]:
trainer.test(datamodule=dm,ckpt_path='best', verbose=True)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.734883725643158     │
│      test_bowel_acc       │    0.9224806427955627     │
│      test_bowel_loss      │    0.27268218994140625    │
│  test_extravasation_acc   │    0.7713178396224976     │
│  test_extravasation_loss  │    0.5381880402565002     │
│      test_kidney_acc      │    0.7713178396224976     │
│     test_kidney_loss      │    0.6896868348121643     │
│      test_liver_acc       │    0.6317829489707947     │
│      test_liver_loss      │    0.8325867056846619     │
│         test_loss         │    3.3029801845550537     │
│      test_spleen_acc      │     0.577519416809082     │
│     test_spleen_loss      │    0.9698363542556763     │
└───────────────────────────┴───────────────────────────┘

[{'test_bowel_loss': 0.27268218994140625,
  'test_extravasation_loss': 0.5381880402565002,
  'test_kidney_loss': 0.6896868348121643,
  'test_liver_loss': 0.8325867056846619,
  'test_spleen_loss': 0.9698363542556763,
  'test_loss': 3.3029801845550537,
  'test_bowel_acc': 0.9224806427955627,
  'test_extravasation_acc': 0.7713178396224976,
  'test_kidney_acc': 0.7713178396224976,
  'test_liver_acc': 0.6317829489707947,
  'test_spleen_acc': 0.577519416809082,
  'test_acc': 0.734883725643158}]